In [236]:
import pandas as pd
import numpy as np
import re
import os

In [237]:
# pd.set_option('display.max_colwidth', None)

In [238]:
pd.reset_option('display.max_colwidth')

In [239]:
master_data = {
    "olx_2023_master" : "../original_data_2023/olx/ALL_DATA/gdansk_olx_master_data.json",
    "olx_2022_master" : "../original_data_2022/olx/ALL_DATA/gdansk_olx22_master_data.json",
}

dataframes_08_23 = {}

for key, value in master_data.items():
    master_data[key] = pd.read_json(value)  

master_22 = master_data["olx_2022_master"]
master_23 = master_data["olx_2023_master"]

In [240]:
master_22.head()

,building_type,collection_set,furnished,level,link,listing_no,negotiable,private_seller,publish_date,rent,rent_extra,rent_full,rooms,surface,title
0,4,1,True,0.0,https://www.olx.pl/d/oferta/pokoj-na-wypoczyne...,1,True,1.0,2022-09-02,120,1,121,1,32,Pokój na wypoczynek na Mazurach
1,4,1,True,8.0,https://www.olx.pl/d/oferta/mieszkanie-3-pokoj...,2,False,NaN,2022-09-03,3200,570,3770,3,46,Mieszkanie 3 pokojowe na Przymorzu dla student...
2,4,1,True,3.0,https://www.olx.pl/d/oferta/zabi-kruk-studio-a...,3,False,1.0,2022-09-03,2500,300,2800,1,20,Żabi Kruk Studio Apartament z Balkonem Kawaler...
3,2,1,True,0.0,https://www.olx.pl/d/oferta/mieszkanie-dwupoko...,4,False,1.0,2022-09-03,2000,260,2260,2,38,Mieszkanie dwupokojowe z ogródkiem wynajmę
4,4,1,True,3.0,https://www.olx.pl/d/oferta/wynajme-2-pokojowe...,5,False,1.0,2022-09-03,2000,450,2450,2,41,Wynajme 2-pokojowe mieszkanie w centrum miasta


In [241]:
master_23.head()

,rent,location,username,on_olx_since,last_activity,add_date,title,link,description,Prywatne,Poziom,Umeblowane,Rodzaj zabudowy,Powierzchnia,Liczba pokoi,Czynsz (dodatkowo),Firmowe
0,2 250 zł,Stogi z Przeróbką,Andrzej,sierpień 2021,Ostatnio online w dniu 18 sierpnia 2023,18 sierpnia 2023,Dodane 18 sierpnia 2023,https://www.olx.pl/d/oferta/sloneczne-dwa-poko...,Mieszkanie położone jest w dzielnicy Przeróbka...,1.0,2,Tak,Blok,38 m²,2 pokoje,750 zł,NaN
1,2 700 zł,Chełm z dzielnicą Gdańsk Południe,Tomasz Pankau,lipiec 2020,Ostatnio online w dniu 18 sierpnia 2023,20 sierpnia 2023,Dodane 20 sierpnia 2023,https://www.olx.pl/d/oferta/mieszkanie-na-wyna...,"Wynajmę 2-pokojowe, w pełni umeblowane mieszka...",1.0,2,Tak,Blok,51 m²,2 pokoje,650 zł,NaN
2,2 800 zł,Siedlce,Justyna,grudzień 2017,Ostatnio online wczoraj o 20:13,20 sierpnia 2023,Dodane 20 sierpnia 2023,https://www.olx.pl/d/oferta/mieszkanie-dwupoko...,Wynajmę mieszkanie 2- pokojowe w centrum Gdańs...,1.0,1,Tak,Apartamentowiec,40 m²,2 pokoje,320 zł,NaN
3,2 900 zł,Przymorze Małe,Anita,październik 2012,Ostatnio online w dniu 14 sierpnia 2023,13 sierpnia 2023,Dodane 13 sierpnia 2023,https://www.olx.pl/d/oferta/mieszkanie-2-pok-a...,"Wynajmę mieszkanie na osiedlu Albatros Tower ,...",1.0,2,Tak,Apartamentowiec,41 m²,2 pokoje,500 zł,NaN
4,350 zł,Przymorze Małe,Bogdan,styczeń 2014,Ostatnio online w dniu 17 sierpnia 2023,17 sierpnia 2023,Dodane 17 sierpnia 2023,https://www.olx.pl/d/oferta/mieszkanie-apartam...,"Mieszkanie w bloku, 2-pokojowe znajdujące się ...",1.0,2,Tak,Blok,45 m²,2 pokoje,350 zł,NaN


In [242]:
df = master_23

In [243]:
df.shape

(2088, 17)

In [244]:
df.dropna(subset=["location", "Rodzaj zabudowy", "Liczba pokoi", "Czynsz (dodatkowo)"])

,rent,location,username,on_olx_since,last_activity,add_date,title,link,description,Prywatne,Poziom,Umeblowane,Rodzaj zabudowy,Powierzchnia,Liczba pokoi,Czynsz (dodatkowo),Firmowe
0,2 250 zł,Stogi z Przeróbką,Andrzej,sierpień 2021,Ostatnio online w dniu 18 sierpnia 2023,18 sierpnia 2023,Dodane 18 sierpnia 2023,https://www.olx.pl/d/oferta/sloneczne-dwa-poko...,Mieszkanie położone jest w dzielnicy Przeróbka...,1.0,2,Tak,Blok,38 m²,2 pokoje,750 zł,NaN
1,2 700 zł,Chełm z dzielnicą Gdańsk Południe,Tomasz Pankau,lipiec 2020,Ostatnio online w dniu 18 sierpnia 2023,20 sierpnia 2023,Dodane 20 sierpnia 2023,https://www.olx.pl/d/oferta/mieszkanie-na-wyna...,"Wynajmę 2-pokojowe, w pełni umeblowane mieszka...",1.0,2,Tak,Blok,51 m²,2 pokoje,650 zł,NaN
2,2 800 zł,Siedlce,Justyna,grudzień 2017,Ostatnio online wczoraj o 20:13,20 sierpnia 2023,Dodane 20 sierpnia 2023,https://www.olx.pl/d/oferta/mieszkanie-dwupoko...,Wynajmę mieszkanie 2- pokojowe w centrum Gdańs...,1.0,1,Tak,Apartamentowiec,40 m²,2 pokoje,320 zł,NaN
3,2 900 zł,Przymorze Małe,Anita,październik 2012,Ostatnio online w dniu 14 sierpnia 2023,13 sierpnia 2023,Dodane 13 sierpnia 2023,https://www.olx.pl/d/oferta/mieszkanie-2-pok-a...,"Wynajmę mieszkanie na osiedlu Albatros Tower ,...",1.0,2,Tak,Apartamentowiec,41 m²,2 pokoje,500 zł,NaN
4,350 zł,Przymorze Małe,Bogdan,styczeń 2014,Ostatnio online w dniu 17 sierpnia 2023,17 sierpnia 2023,Dodane 17 sierpnia 2023,https://www.olx.pl/d/oferta/mieszkanie-apartam...,"Mieszkanie w bloku, 2-pokojowe znajdujące się ...",1.0,2,Tak,Blok,45 m²,2 pokoje,350 zł,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,2 990 zł,"Gdańsk, Letnica\nPomorskie",Aleksandra,maj 2016,Ostatnio online dziś o 16:30,27 września 2023,Mieszkanie Letnica z dostępem do siłowni i sauny,https://www.olx.pl/d/oferta/mieszkanie-letnica...,"Na wynajem nowoczesne mieszkanie, blisko plaży...",1.0,2,Tak,Apartamentowiec,50 m²,2 pokoje,700 zł,NaN
2084,2 100 zł,"Gdańsk, Śródmieście\nPomorskie",MIROSŁAW,czerwiec 2016,Ostatnio online dziś o 05:44,08 października 2023,Wynajem mieszkania,https://www.olx.pl/d/oferta/wynajem-mieszkania...,Urokliwe mieszkanie na Starówce Gdańskiej przy...,NaN,2,Tak,Kamienica,33 m²,Kawalerka,550 zł,1.0
2085,2 600 zł\n do negocjacji,"Gdańsk, Przymorze Wielkie\nPomorskie",Janusz,wrzesień 2015,Ostatnio online w dniu 05 października 2023,03 października 2023,Mieszkanie 2-4 osob. (od października do końca...,https://www.olx.pl/d/oferta/mieszkanie-2-4-oso...,fertę wynajmu kieruję do studentów.\n\nNa wyna...,1.0,4,Tak,Blok,43 m²,2 pokoje,800 zł,NaN
2086,3 000 zł,"Gdańsk, Brzeźno\nPomorskie",Q4 Apartments,marzec 2019,Ostatnio online dziś o 10:01,12 września 2023,Mieszkanie do wynajęcia 2 pokoje Gdańsk-Brzeźno,https://www.olx.pl/d/oferta/mieszkanie-do-wyna...,ddany w 2020 roku apartament o podwyższonym st...,NaN,2,Tak,Blok,47 m²,2 pokoje,550 zł,1.0


In [245]:
df.isna().sum()

rent                     0
location                 1
username                 0
on_olx_since             0
last_activity            0
add_date                 0
title                    0
link                     0
description              0
Prywatne               275
Poziom                  44
Umeblowane               0
Rodzaj zabudowy          1
Powierzchnia             1
Liczba pokoi             1
Czynsz (dodatkowo)       3
Firmowe               1813
dtype: int64

In [246]:
olx_gda_districts_mapping = {
    "Śródmieście": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "Przymorze Wielkie": "Przymorze",
    "Brzeźno": "Brzezno",
    "Przymorze Małe": "Przymorze",
    "Wrzeszcz": "Wrzeszcz",
    "Żabianka - Wejhera - Jelitkowo - Tysiąclecia": "Zabianka - Wejhera - Jelitkowo - Tysiaclecia",
    "Chełm z dzielnicą Gdańsk Południe": "Chelm - Lostowice - Poludnie",
    "Jasień": "Jasien - Matarnia - Kokoszki - Osowa",
    "Ujeścisko - Łostowice": "Chelm - Lostowice - Poludnie",
    "Letnica": "Letnica",
    "Piecki-Migowo": "Piecki Migowo - Suchanino - Morena",
    "Zaspa Młyniec": "Zaspa",
    "Orunia - Św. Wojciech - Lipce": "Orunia",
    "Zaspa Rozstaje": "Zaspa",
    "Siedlce": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    "Stogi z Przeróbką": "Przerobka - Stogi - Nowy Port",
    "Oliwa": "Oliwa - Strzyza",
    "Aniołki": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    "Wzgórze Mickiewicza": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    "Matarnia": "Jasien - Matarnia - Kokoszki - Osowa",
    "Suchanino": "Piecki Migowo - Suchanino - Morena",
    "Brętowo": "Piecki Migowo - Suchanino - Morena",
    "Kokoszki": "Jasien - Matarnia - Kokoszki - Osowa",
    "Strzyża": "Oliwa - Strzyza",
    "Młyniska": "Siedlce - Aniolki - Mlyniska - Mickiewicza"
}

olx_rooms_mapping = {
    "2pokoje": 2,
    "kawalerka": 1,
    "3pokoje": 3, 
    "4iwięcej": 4,

}

olx_private_seller_mapping = {
    1.0: True,
    np.nan: False
}

olx_furnished_mapping = {
    "Tak": True,
}

olx_level_mapping = {
    "powyżej10": 11,
    "suterena": 0,
    "parter": 0,
    "1":1, "2": 2, "3":3, "4":4, "5":5, "6":6, "7":7, "8":8, "9":9, "10":10
}

olx_building_type_mapping = {
    "blok": "unit",
    "apartamentowiec": "apartment_building",
    "kamienica": "tenement",
    "dom wolnostojący": "house",
    "szeregowiec": "house",
    "loft": "unit"
}

In [247]:
def convert_numeric_to_float(convertable_string):
    return float(convertable_string.replace("zł", "").replace(" ", "").replace(",",".").replace("\ndonegocjacji", ""))

def clear_olx(df: pd.DataFrame, city):
    df.dropna(subset=["location", "Rodzaj zabudowy", "Liczba pokoi", "Czynsz (dodatkowo)"], inplace=True)
    
    df.drop(
        labels=[
            'link', 
            'username', 
            'on_olx_since', 
            'last_activity',
            'title',
            'description',
            'Firmowe',
            'add_date',
        ],
        inplace= True,
        axis= 1
    )
    
    is_negotiable_list = []
    is_furnished_list = []
    
    for index, row in df.iterrows():

        # negotiable
        if 'do negocjacji' in row['rent']:
            is_negotiable_list.append(True)
        else:
            is_negotiable_list.append(False)

        # furnished
        if 'Tak' in row['Umeblowane']:
            is_furnished_list.append(True)
        else:
            is_furnished_list.append(False)

    # negotiable apply
    df['negotiable'] = is_negotiable_list

    # building_type
    df = df.rename(mapper={"Rodzaj zabudowy": "building_type"}, axis=1)
    df["building_type"] = df["building_type"].str.replace(" ", "").str.lower()
    df["building_type"] = df["building_type"].map(olx_building_type_mapping)
    
    # furnished apply
    df = df.rename(mapper={"Umeblowane": "furnished"}, axis=1)
    df['furnished'] = is_furnished_list
    
    # private_seller
    df = df.rename(mapper={"Prywatne": "private_seller"}, axis=1)
    df['private_seller'] = df['private_seller'].map(olx_private_seller_mapping)
    
    # negotiable apply
    df['negotiable'] = is_negotiable_list
    
    # surface
    df = df.rename(mapper={"Powierzchnia": "surface"}, axis=1)
    df['surface'] = df['surface'].str.replace("m²", "").str.replace(" ", "").str.replace(",", ".").astype(float)

    # level 
    df = df.rename(mapper={"Poziom": "level"}, axis=1)
    df['level'] = df['level'].str.replace(" ", "").str.lower()
    df['level'] = df['level'].map(olx_level_mapping)
    df['level'].fillna(1001, inplace=True)
    
    # rooms 
    df = df.rename(mapper={"Liczba pokoi": "rooms"}, axis=1)
    df['rooms'] = df['rooms'].str.replace(" ", "").str.lower()
    df['rooms'] = df['rooms'].map(olx_rooms_mapping)
    df['rooms'] = df['rooms'].astype(int)
    
    # rent
    df['rent'] = df['rent'].apply(convert_numeric_to_float)

    # rent_extra
    df = df.rename(mapper={"Czynsz (dodatkowo)": "rent_extra"}, axis=1)
    df['rent_extra'] = df['rent_extra'].apply(convert_numeric_to_float)

    # location
    if city.lower() == "gda":
        df['location'] = df['location'].map(olx_gda_districts_mapping)
    elif city.lower() == "krk":
        pass
        # df['location'] = df['location'].map(olx_krk_districts_mapping)

    return df

In [248]:
df = clear_olx(df, city="gda")

In [249]:
df["location"].value_counts()

location
Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody    388
Przymorze                                                    260
Chelm - Lostowice - Poludnie                                 129
Wrzeszcz                                                     124
Brzezno                                                      123
Zabianka - Wejhera - Jelitkowo - Tysiaclecia                 105
Siedlce - Aniolki - Mlyniska - Mickiewicza                    84
Jasien - Matarnia - Kokoszki - Osowa                          79
Zaspa                                                         66
Piecki Migowo - Suchanino - Morena                            58
Letnica                                                       57
Oliwa - Strzyza                                               36
Orunia                                                        30
Przerobka - Stogi - Nowy Port                                 22
Name: count, dtype: int64

In [260]:
df22 = master_22
df23 = df
df22.drop(labels=["collection_set", "listing_no", "private_seller", "rent_full"], axis=1, inplace=True)

KeyError: "['collection_set', 'listing_no', 'private_seller', 'rent_full'] not found in axis"

In [252]:
similar_locations = [
    "Siedlce - Aniolki - Mlyniska - Mickiewicza", 
    "Chelm - Lostowice - Poludnie", 
    "Jasien - Matarnia - Kokoszki - Osowa",
    "Oliwa - Strzyza",
    "Przerobka - Stogi - Nowy Port", 
    "Orunia"]

In [257]:
df23 = df23[(df23.rooms == 1) & 
        (df23.rent <= 5000) & 
        (df23.rent >= 1200) & 
        (df23.building_type == "unit") &
        (df23.rent_extra <= 1000)]

df23.describe()
# df23 = df23[ df23.location.isin(similar_locations) ]

,rent,level,surface,rooms,rent_extra
count,209.000000,209.000000,209.000000,209.0,209.000000
mean,2361.004785,50.440191,28.790287,1.0,465.093110
std,369.691198,213.605850,5.397716,0.0,147.742031
min,1700.000000,0.000000,16.000000,1.0,1.000000
25%,2100.000000,1.000000,26.000000,1.0,380.000000
50%,2300.000000,2.000000,29.000000,1.0,450.000000
75%,2500.000000,4.000000,32.000000,1.0,550.000000
max,4150.000000,1001.000000,50.000000,1.0,1000.000000


In [255]:
df22 = df22[(df22.rooms == 1) & 
            (df22.rent <= 5000) & 
            (df22.rent >= 1000) & 
            (df22.building_type == 4) & 
            (df22.rent_extra <= 1000)]
df22.describe()

,building_type,level,rent,rent_extra,rooms,surface
count,198.0,189.000000,198.000000,198.000000,198.0,198.000000
mean,4.0,3.216931,2181.045455,381.762626,1.0,28.494949
std,0.0,2.570398,317.664163,105.496069,0.0,5.539656
min,4.0,0.000000,1150.000000,1.000000,1.0,8.000000
25%,4.0,1.000000,2000.000000,330.000000,1.0,25.000000
50%,4.0,3.000000,2200.000000,380.000000,1.0,30.000000
75%,4.0,4.000000,2400.000000,450.000000,1.0,32.000000
max,4.0,11.000000,3270.000000,650.000000,1.0,45.000000


In [258]:
2361/2181*100

108.25309491059147